# TME 5 : Projet Detection de motifs


## Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [1]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence



def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence ="".join([nuc[random.randint(0,3)] for i in range(n)])
    if upper:
        return sequence.upper()
    return sequence.lower()


def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = ''
    while nbpos>0:
        rand=random.randint(0,len(motif)-1)
        motifM=motif[:rand]+nuc[(nuc.index(motif[rand])+random.randint(1,3))%len(nuc)]+motif[rand+1:]
        nbpos-=1
    if upper:
        return motifM.upper()
    return motifM.lower()


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
    
  
    sequences = []
    for i in range(t):
        s=generateRandomSequence(n-k, True)
        motif=modifierMotif(generateRandomSequence(k, True), v,True)
        i=random.randint(0,len(s)-1)
        s=s[:i]+motif+s[i:]
        sequences+=[s]
    
    return sequences

adn = implantMotifVar(k, v, t, n)
print (adn)


['TTCGTGTTGT', 'CCTCTGATTT', 'ATTCCGAACA']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [2]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
print (adn)

['CGATAACGAC', 'GGATCAGATG', 'CACGCGTTCA', 'CACATAAGCT', 'TATATGCGTA', 'ACGAAAGTAT', 'ACAGAGCGCC', 'GAGATTACGA', 'TCCTCCACTT', 'TGTTTTTTAG']


3\. Préparation pour l'algorithme _"Brute Force Motif Search"_. Avant d'implementer  l'algorithme _"Brute Force Motif Search"_, faites une fonction pour calculer le score `score(s, seqs)`, où `s` est un ensemble de position initiales `s`$=(s_1,s_2,...,s_k)$  et `seqs` est une matrice de taille `t x n`  (`t` est le nombre de séquences et `n` la taille de ces séquences), pour tester votre fonction utiliser la matrice et les valeurs ci-dessous (voir aussi les slides du cours).


In [3]:
import numpy as np

k =8
n = 68 # length of each DNA sequence
t = 5 # number of  DNA sequence samples

s=(25,20,2,55,59)

adn = [
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]



def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn avec les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs
    """
    motifs = []
    for i in range(len(seqs)):
        m=seqs[i][s[i]:s[i]+k]
        if (len(m) == k):
            motifs.append(m)
            
    return motifs


def profile(motifs, k, nuc):
    """
    Construire une PWM (position weight matrix)
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie PWM: position weight matrix
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    for m in motifs:
        for i in range(k):
            PWM[nuc.index(m[i])][i]+=1
    return PWM


def score(prof, k):
    """
    Calcul le score d'un PWM, la somme de max de chaque colonne
    entrée PWM: position weight matrix
    entrée k: taille du motif
    sortie score: la somme de max de chaque colonne
    """
    sc = 0
    for i in range(k):
        m=0
        for l in prof:
            if l[i] > m:
                m=l[i]
        sc+=m
    return sc

motifs = extractSeqs(s, adn, k)
PWM = profile(motifs, k, nuc)
print(PWM)
sc = score(PWM, k)
#teste, sc doit etre egalle à 30
print (sc)



[[3. 0. 1. 0. 3. 1. 1. 0.]
 [2. 4. 0. 0. 1. 4. 0. 0.]
 [0. 1. 4. 0. 0. 0. 3. 1.]
 [0. 0. 0. 5. 1. 0. 1. 4.]]
30.0


4\. Implementer l'algoritme _"Brute Force Motif Search"_ pour chercher des motifs de taille variable. Attention, la complexité de cet algorithme est trop importante, car il enumère toutes les positions intiales possibles, nous allons donc le tester seulement sur de petits jeux de données.

In [12]:
from itertools import product


#enumeration all initial positions
def initialPositions(k, n, t):
    """
    Génère des vecteurs avec tous les positions de départ possible
    entrée k: taille du motif
    entrée n : longueur des séquences 
    entrée t : nombre de séquences 
    """
    allIniPos = [];
    
    for i in range(n):
        allIniPos.append(i)
    return [e for e in product(allIniPos,repeat=t)]

def bruteForceMotifSearch(sequences, t, n, k):
    """
    Implémente l'algorithme bruteForceMotifSearch 
    entrée séquences: matrice de dimension txn avec les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k: taille du motif 
    sortie bestMotif: vecteur de positions de départ contenant le motif de meilleur score 
    sortie bestscore: score associé au meilleur motif
    """
    bestscore = 0; bestMotif=''
    allIniPos = initialPositions(k, n, t)
    for v in allIniPos:
        motifs=extractSeqs(v, sequences, k)
        if (score(profile(motifs,k,nuc),k)>bestscore):
            bestscore=score(profile(motifs,k,nuc),k)
            bestMotif=v
        
    return bestMotif, bestscore

#test
k=4; v=1; t=3; n=10
sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)
print (bestMotif, bestscore)
print (extractSeqs(bestMotif, sequences, k))

['ACCTACTCGC', 'GGACAACCAG', 'TTGTATTGTT']
(2, 0, 2) 9.0
['CTAC', 'GGAC', 'GTAT']


5\. L'algorithme est-il performant sur ce jeu de données ? Sinon, pourquoi ?

<font color='blue'>
reponse: non, la longeur de ce jeu de données est trop faible.
</font>

6\. Tester l'agorithme avec les paramètres suivants `k=7`, `v=1`, `t=5`, et `n=20`. Avez vous de meilleurs performances ?

In [5]:
k=7; v=1; t=5; n=20

sequences = implantMotifVar(k, v, t, n)
print (sequences)
bestMotif, bestscore = bruteForceMotifSearch(sequences, t, n, k)

print (bestMotif)
print (extractSeqs(bestMotif, sequences, k))

['CTCGCACAGTTGGGGCAGTG', 'AAGTGATTTGGCGGTAACCG', 'AACAAAGAGTTACAACGACG', 'TTTCTGGAGTTGTGCTCGGC', 'CTGAGGCCCCGGGCCTTAGC']
(6, 4, 6, 6, 12)
['CAGTTGG', 'GATTTGG', 'GAGTTAC', 'GAGTTGT', 'GCCTTAG']


7\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les mêmes jeux de données que la question 3 (voir slides de cours). Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles.

In [6]:
#Genere tous les K-mers de taille K ayant de AAA... à TTT...
allkmers = product(nuc, repeat=k)
kmers=["".join(k) for k in list(allkmers)]
print(len(kmers))
reLow=5

def removeLowComplexePair(motifs):
    """
    Eleve les motifs peu complexe 
    entrée motifs: liste de motifs 
    sortie validMotif: liste de motifs filtré
    """
    validMotif = []
    for mo in motifs:
        k=0
        cpt=0
        for i in range(1,len (mo)-reLow+2):
            k=1
            cpt=1
            while( cpt/2<=2 and k<=reLow  ):
                if i+k-1<len(mo) and mo[i+k-2]==mo[i+k-1]:
                    k+=1
                elif i+cpt<len(mo) and mo[i+cpt-2]==mo[i+cpt]:
                    cpt+=1
                else:
                    break  
            if k>=reLow or cpt/2>=2:#repetition de lettres superieures a m
                break #motif suivant
        if k<reLow and cpt/2<2:#si aucune rep
            validMotif.append(mo) #add
            
    return validMotif


#Enlever les motifs peu complexe
kmersValid = removeLowComplexePair(kmers)
print (len(kmersValid))




16384
15744


In [7]:
def hamdist(str1, str2):
    """
    Calcul la distance de hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de hamming
    """
    distance = 0
    for i in range(len(str1)):
        if (str1[i] != str2[i]) :
            distance += 1
    return distance


def totalDistance(motif, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: sequence consensus
    entrée sequences: chaîne de caractères
    entrée k: taille du motif 
    entrée sequences: matrice de dimension txn contenant les séquences
    sortie td: somme de distance de hamming minimal
    """
    td = 0
    for s in sequences:
        Min=hamdist(motif,s[:k])
        for i in range(len(s) - k + 1) :
            tmp = hamdist(motif, s[i:i+k])
            if (tmp < Min) : 
                Min = tmp
        td +=Min       
    return td


def MedianStringSearch(allkmers, sequences, t, n, k):
    """
    Implement l'algorithme MedianStringSearch
    entrée allkmers: liste de K-mers valides
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée n : longueur des séquences 
    entrée t : nombre de séquences 
    entrée k: taille du motif 
    sortie bestMotif: le motif que minimise les distances
    sortie bestDistance: la distance minimal
    sortie motifDist: un dictionnaire contenant les motifs et leurs distances
    REMARK: n peut varié dans les fichiers de chip-seq, vous pouvez donc l'extraire de matrice sequence 
    """
    bestDistance = 1000
    bestMotif = ''
    motifDist = {}
    
    for kmer in allkmers : 
        td = totalDistance(kmer, sequences, k)
        if td < bestDistance :
            bestDistance = td
            bestMotif = kmer
            
        motifDist[kmer] = td
        
    return bestMotif, bestDistance, motifDist

bestMotif, bestDistance, motifDist = MedianStringSearch(kmersValid, sequences, t, n, k)



In [8]:
#Apliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

#Faire un merge de motifs trouvés dans les deux brin

def mergeMotifs(motifs, motifsRevComp):
    ''' 
    Merge motifs trouvé dans les brins sens et complementaire
    entrée motifs : un dictionnaire contenant les motifs du brin sens et leurs distances
    entrée motifsRevComp : un dictionnaire contenant les motifs du brin complementaire et leurs distances
    sortie allMotif: merge de motifs et motifsRevComp
    '''
    
    allMotif = {}
    allMotif=motifs | motifsRevComp
    return allMotif


#Faire une fonction pour afficher les top meilleurs motifs (distance minimal)

def printTopFMotifsFreq(motifDist, top, rev=False):
    """
    Afficher les top motifs avec les plus petit distances
    entrée motifDist: un dictionnaire contenant les motifs et leurs distances
    entrée top: max de motifs à afficher
    sortie : None
    """
    print(dict([(k,v) for k,v in sorted(motifDist.items(),key=lambda x: x[1],reverse=rev)][len(motifDist)-top:]))
        


printTopFMotifsFreq(motifDist, 6, True)
revSequences = "" #sequences du brin complementaire 

bestMotifRev, bestDistanceRev, motifDistRev = MedianStringSearch(kmersValid, revSequences, t, n, k)
print (bestMotifRev, bestDistanceRev)

printTopFMotifsFreq(motifDistRev, 6, True)

allmotif = mergeMotifs(motifDist, motifDistRev)
print ('mergerd')
printTopFMotifsFreq(allmotif, 6, True)

{'GGAGTTA': 9, 'GGAGTTG': 9, 'GGCGTTA': 9, 'GGGAGTT': 9, 'TGAGTTG': 9, 'GAGTTGG': 8}
AAAACAA 0
{'TTTTGGG': 0, 'TTTTGGT': 0, 'TTTTGTA': 0, 'TTTTGTC': 0, 'TTTTGTG': 0, 'TTTTGTT': 0}
mergerd
{'TTTTGGG': 0, 'TTTTGGT': 0, 'TTTTGTA': 0, 'TTTTGTC': 0, 'TTTTGTG': 0, 'TTTTGTT': 0}


8\. Comparation entre les algorithmes _"Median String Search"_ et "_Brute Force Motif Search_"

a\. Produire des séquences artificielles avec les paramètres suivants  `k=7`, `v=1`, `t=5`, `n=20`. Comparer les performance et le temps de calculs des deux algorithmes.

In [9]:
k=7; v=1; t=5; n=20

b\. Quel algorithme peut etre appliqué au probleme de recherche de motifs? Et comment?

<font color='blue'>
   L'algorithme à utiliser est le Median String search. 
</font>

In [11]:
n = 100

def readFasta(fastaFile, n):
    """
    Lire n sequences d'un ficher fasta
    entrée fastaFile: nom du fichier
    entrée n: nombre de sequences à retourner
    sortie sequence: liste de sequences
    """
    sequence = []
    file = open(fastaFile, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    return sequence

peaksFile = "Sequence_by_Peaks_8.fasta" #votre fichier de Peaks


sequences   = readFasta(peaksFile, n)
print(sequences)


['...']
